## Notebook to create brain state files from EF1_ALPHA excel sleep scoring spreadsheet

In [166]:
import os 
import numpy as np
import pandas as pd

In [167]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA')
epoch_number = "A, E, I, M, Q, U, Y, AC, AG, AK, AO, AS, BA, BE, BI, BM, BQ, BU, BY, CC, CG, CK, CO, CS, CW"
cols = "B, F, J, N, R, V, Z, AD, AH, AL, AP, AT, BB, BF, BJ, BN, BR, BV, BZ,CD, CH, CL, CP, CT, CX"
cols_discard = "C, G, K, O, S, W, AA, AE, AI, AM, AQ, AU, BC, BG, BK, BO, BS, BW, CA, CE, CI, CM, CQ, CU, CY"
skiprows = 5
raw_excel = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header= 1)

In [168]:
#list of animal ids
column_names = list(raw_excel)
animal_ids = [anim_id[0:10] for anim_id in column_names]

In [169]:
#strip whitespace
animal_ids = [id.replace(" ", "") for id in animal_ids]
print(animal_ids)

['210422_D', '210423_D', '210705_C', '210706_C', '191125_A', '191126_A', '210705_B', '210706_B', '210705_A', '210706_A', '191216_A_1', '191217_A_1', '191104_B', '191107_A', '191108_A', '210422_B_1', '210423_B_1', '191216_C', '191217_C', '191216_B', '191217_B', '210429_C', '210430_C', '210705_D', '210706_D']


In [67]:
raw_epochs = pd.read_excel('grant_EEG_classification.xlsx', usecols = epoch_number, header = 1, skiprows = 6)
raw_epochs_letter = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols, header = 1, skiprows = 6)
raw_discard = pd.read_excel('grant_EEG_classification.xlsx', usecols = cols_discard, header = 1, skiprows = 6)

In [68]:
two_files = []
for epoch in raw_epochs:
    start_epochs = raw_epochs[raw_epochs[epoch] == 0].index.tolist()
    two_files.append(start_epochs)

In [141]:
for file, epoch, animal_id, epoch_letter_col, discard_col in zip(two_files, raw_epochs, animal_ids, raw_epochs_letter, raw_discard):
    #print(animal_id)
    if len(file) == 2:
        start_2 = file[1:2]
        start_2 = start_2[0]
        end_epoch = start_2 - 1
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        part_1 = epoch_column_raw[0: end_epoch].astype(int)
        part_2 = epoch_column_raw[start_2: length_col].astype(int)
        epochs_letter_part_1 = epochs_letter[0:end_epoch]
        epochs_letter_part_2 = epochs_letter[start_2: length_col]
        discard_data = raw_discard[discard_col]
        discard_part_1 = discard_data[0: end_epoch]
        discard_part_2 = discard_data[start_2: length_col]
        part_1_dict = {'epoch_numbers': part_1, 'brain_state': epochs_letter_part_1,
                       'epoch_discard_numbers': discard_part_1}
        part_2_dict = {'epoch_numbers': part_2, 'brain_state': epochs_letter_part_2,
                      'epoch_discard_numbers': discard_part_2}
        df_1 = pd.DataFrame(data = part_1_dict)
        df_2 = pd.DataFrame(data = part_2_dict)
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        #df_1.to_csv(str(animal_id) +'_part_1.csv', index=True) 
        #df_2.to_csv(str(animal_id) +'_part_2.csv', index=True)
    else:
        epochs_letter = raw_epochs_letter[epoch_letter_col]
        drop_na = epochs_letter.dropna()
        length_col = len(drop_na) 
        epoch_column_raw = raw_epochs[epoch]
        all_epochs = epoch_column_raw[0:length_col].astype(int)
        all_epochs_letter = epochs_letter[0:length_col]
        discard_data = raw_discard[discard_col]
        all_discard_epochs = discard_data[0:length_col] 
        data_dict = {'epoch_numbers': all_epochs, 'brain_state': all_epochs_letter,
                       'epoch_discard_numbers': all_discard_epochs}
        os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
        data_df = pd.DataFrame(data = data_dict)
        #data_df.to_csv(str(animal_id) +'.csv', index=True)

In [ ]:
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder')
test_check = pd.read_csv('191125_A_part_2.csv')

In [ ]:
wake_indices = test_check.loc[test_check['brain_state'] == 'W'].index.tolist()
rem_indices = test_check.loc[test_check['brain_state'] == 'R']
nonrem_indices = test_check.loc[test_check['brain_state'] == 'N']
discard_indices = test_check.loc[test_check['epoch_discard_numbers'] == 'E']

In [ ]:
wake_indices_list = wake_indices.index.tolist()
rem_indices_list = rem_indices.index.tolist()
nonrem_indices_list = nonrem_indices.index.tolist()
discard_indices_list = discard_indices.index.tolist()

In [ ]:
def non_match_elements(list_a, list_b):
    non_match = []
    for i in list_a:
        if i not in list_b:
            non_match.append(i)
    return non_match

new_list = non_match_elements(wake_indices_list, discard_indices_list)

## check that code works on individual animals

In [12]:
import os
os.chdir('/home/melissa/PROJECT_DIRECTORIES/ef1_alpha_analysis')
%run preprocess.py
%run ef1_alpha_properties.py
%run filter.py
%run power.py

In [7]:
#paths 
recording_path = '/home/melissa/PREPROCESSING/EF1_ALPHA'
brain_state_path = '/home/melissa/PREPROCESSING/EF1_ALPHA/brain_state_folder'

#paths to folder 1 and 2 
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/two_recording/Part_1')
test1 = np.load('210705_part1_B.npy')
os.chdir('/home/melissa/PREPROCESSING/EF1_ALPHA/two_recording/Part_2')
test2 = np.load('210705_part2_B.npy')

In [4]:
#2 lists of letters - 1 for one recording and 1 for two recording
recordings_letters_1 = ['A', 'A_1', 'B', 'C', 'D']
recording_letters_2 = ['A', 'A_1', 'B', 'B_1', 'C', 'D']

In [5]:
brainstates = ['R']
animal_id_not_running = {'A': ['191125']}
hdstage_not_running = ['A']

In [23]:
def get_seizure_epochs(brain_state_file):

    seizure_indices = brain_state_file.loc[brain_state_file['epoch_discard_numbers'] == 'E'].index.tolist()
        
    return seizure_indices

In [1]:
#get_seizure_epochs(brain_state_file = br_file_1)